# Прогнозирование продаж и отгрузок для маркетплейсов

Этот notebook содержит полный цикл прогнозирования продаж и расчета отгрузок.


## 1. Импорт библиотек и настройка


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Добавляем путь к модулю forecasting
sys.path.insert(0, str(Path().resolve().parent.parent))

from forecasting.forecaster import SalesForecaster
from forecasting.data_loader import DataLoader
from forecasting.shipment_calculator import ShipmentCalculator

# Настройка отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


## 2. Загрузка данных


In [ ]:
# Путь к данным
data_path = '../data'

# Создание загрузчика данных
loader = DataLoader(data_path)

# Загрузка всех данных
data = loader.load_all_data()

# Загрузка исторических отгрузок
historical_shipments = loader.load_historical_shipments('Отгрузки в МП')

print("Загруженные данные:")
for key, df in data.items():
    if df is not None and not df.empty:
        print(f"  {key}: {len(df)} записей")
    else:
        print(f"  {key}: нет данных")

if historical_shipments is not None and not historical_shipments.empty:
    print(f"\nИсторические отгрузки: {len(historical_shipments)} записей")
    print(historical_shipments.head())


## 3. Анализ исторических данных отгрузок


In [ ]:
if historical_shipments is not None and not historical_shipments.empty:
    # Анализ связи между продажами и отгрузками
    shipment_calc = ShipmentCalculator()
    
    # Анализ для Wildberries
    if 'wb_sales' in data and not data['wb_sales'].empty:
        print("Анализ отгрузок для Wildberries:")
        wb_analysis = shipment_calc.analyze_shipment_calculation(
            data['wb_sales'],
            data.get('wb_stocks', pd.DataFrame()),
            historical_shipments
        )
        print(wb_analysis)
    
    # Анализ для Ozon
    if 'ozon_sales' in data and not data['ozon_sales'].empty:
        print("\nАнализ отгрузок для Ozon:")
        ozon_analysis = shipment_calc.analyze_shipment_calculation(
            data['ozon_sales'],
            data.get('ozon_stocks', pd.DataFrame()),
            historical_shipments
        )
        print(ozon_analysis)
    
    # Визуализация отгрузок
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    # График отгрузок по времени
    shipments_by_date = historical_shipments.groupby('date')['quantity'].sum()
    axes[0].plot(shipments_by_date.index, shipments_by_date.values)
    axes[0].set_title('Исторические отгрузки по времени')
    axes[0].set_xlabel('Дата')
    axes[0].set_ylabel('Количество упаковок')
    axes[0].grid(True)
    
    # Топ продуктов по отгрузкам
    top_products = historical_shipments.groupby('unified_code')['quantity'].sum().sort_values(ascending=False).head(10)
    axes[1].barh(range(len(top_products)), top_products.values)
    axes[1].set_yticks(range(len(top_products)))
    axes[1].set_yticklabels(top_products.index)
    axes[1].set_title('Топ-10 продуктов по отгрузкам')
    axes[1].set_xlabel('Количество упаковок')
    
    plt.tight_layout()
    plt.show()


## 4. Настройка параметров прогнозирования


In [ ]:
# Создание прогнозировщика
forecaster = SalesForecaster(
    data_path=data_path,
    forecast_months=18
)

# Загрузка данных в прогнозировщик
forecaster.data = data
forecaster.data['historical_shipments'] = historical_shipments

# Настройка коэффициента покрытия (можно настроить на основе анализа)
coverage_coefficient = 1.5  # По умолчанию, можно изменить на основе анализа
if historical_shipments is not None and not historical_shipments.empty:
    # Если есть анализ, можно использовать его для настройки
    # Проверяем, был ли выполнен анализ для WB
    if 'wb_sales' in data and not data['wb_sales'].empty:
        shipment_calc = ShipmentCalculator()
        wb_analysis = shipment_calc.analyze_shipment_calculation(
            data['wb_sales'],
            data.get('wb_stocks', pd.DataFrame()),
            historical_shipments
        )
        if wb_analysis:
            if 'avg_coverage_from_shipments' in wb_analysis:
                coverage_coefficient = wb_analysis['avg_coverage_from_shipments']
                print(f"Коэффициент покрытия рассчитан на основе исторических данных: {coverage_coefficient:.2f}")
            elif 'avg_coverage_ratio' in wb_analysis:
                coverage_coefficient = wb_analysis['avg_coverage_ratio']
                print(f"Коэффициент покрытия рассчитан на основе остатков: {coverage_coefficient:.2f}")

forecaster.shipment_calculator.set_coverage_coefficient(coverage_coefficient)
print(f"Используемый коэффициент покрытия: {coverage_coefficient}")

# Размеры коробов
box_sizes = {
    'default': 24  # По умолчанию 24 штуки в коробе
}
forecaster.constraints.set_box_sizes(box_sizes)
print(f"Размер короба по умолчанию: {box_sizes['default']} шт.")

# Даты черной пятницы
ozon_bf = ['2023-11-24', '2024-11-29', '2025-11-28']
wb_bf = ['2023-11-24', '2024-11-29', '2025-11-28']
forecaster.calendar_features.add_black_friday_dates(ozon_bf, wb_bf)
print(f"Черная пятница настроена")

# Подготовка моделей
forecaster.prepare_models()
print(f"\nПодготовлено {len(forecaster.models)} моделей")


## 5. Прогнозирование продаж для Wildberries


In [ ]:
# Прогнозирование всеми моделями
wb_forecasts = forecaster.forecast_sales(marketplace='wb', evaluate=True)

print(f"Создано прогнозов: {len(wb_forecasts)}")
for model_name, forecast_df in wb_forecasts.items():
    print(f"  {model_name}: {len(forecast_df)} записей")


## 6. Выбор лучших прогнозов и применение ограничений


In [ ]:
# Выбор лучших прогнозов для каждого продукта
wb_best_forecast = forecaster.select_best_forecasts(wb_forecasts, marketplace='wb')

print(f"Лучший прогноз: {len(wb_best_forecast)} записей")
print(f"Уникальных продуктов: {wb_best_forecast['unified_code'].nunique()}")
print(f"\nРаспределение по моделям:")
print(wb_best_forecast['selected_model'].value_counts())

# Применение ограничений
wb_best_forecast = forecaster.apply_constraints_to_forecast(wb_best_forecast)

print(f"\nОбщий прогноз продаж: {wb_best_forecast['quantity'].sum():.0f} шт.")


## 7. Расчет отгрузок для Wildberries


In [ ]:
# Расчет отгрузок
wb_shipments = forecaster.calculate_shipments(wb_best_forecast, marketplace='wb')

print(f"Отгрузки: {len(wb_shipments)} записей")
if not wb_shipments.empty:
    print(f"Уникальных складов: {wb_shipments['warehouse'].nunique()}")
    print(f"Уникальных продуктов: {wb_shipments['unified_code'].nunique()}")
    print(f"Общая отгрузка: {wb_shipments['shipment'].sum():.0f} шт.")
    print(f"\nТоп-10 продуктов по отгрузкам:")
    top_shipments = wb_shipments.groupby('unified_code')['shipment'].sum().sort_values(ascending=False).head(10)
    print(top_shipments)
    
    # Визуализация отгрузок
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    # Отгрузки по времени
    shipments_by_date = wb_shipments.groupby('date')['shipment'].sum()
    axes[0].plot(shipments_by_date.index, shipments_by_date.values, marker='o')
    axes[0].set_title('Прогнозные отгрузки по времени (WB)')
    axes[0].set_xlabel('Дата')
    axes[0].set_ylabel('Количество упаковок')
    axes[0].grid(True)
    
    # Отгрузки по складам
    shipments_by_warehouse = wb_shipments.groupby('warehouse')['shipment'].sum().sort_values(ascending=False)
    axes[1].barh(range(len(shipments_by_warehouse)), shipments_by_warehouse.values)
    axes[1].set_yticks(range(len(shipments_by_warehouse)))
    axes[1].set_yticklabels(shipments_by_warehouse.index)
    axes[1].set_title('Отгрузки по складам (WB)')
    axes[1].set_xlabel('Количество упаковок')
    
    plt.tight_layout()
    plt.show()
else:
    print("Нет данных об отгрузках")


## 8. Прогнозирование для Ozon


In [ ]:
# Прогнозирование для Ozon
ozon_forecasts = forecaster.forecast_sales(marketplace='ozon', evaluate=True)
ozon_best_forecast = forecaster.select_best_forecasts(ozon_forecasts, marketplace='ozon')
ozon_best_forecast = forecaster.apply_constraints_to_forecast(ozon_best_forecast)
ozon_shipments = forecaster.calculate_shipments(ozon_best_forecast, marketplace='ozon')

print(f"Ozon - Прогноз продаж: {ozon_best_forecast['quantity'].sum():.0f} шт.")
if not ozon_shipments.empty:
    print(f"Ozon - Отгрузки: {ozon_shipments['shipment'].sum():.0f} шт.")
    print(f"Ozon - Складов: {ozon_shipments['warehouse'].nunique()}")


## 9. Оценка моделей


In [ ]:
# Сводка по оценке моделей
evaluation_summary = forecaster.evaluator.get_evaluation_summary()
best_models_summary = forecaster.evaluator.get_best_models_summary()

if not evaluation_summary.empty:
    print("Оценка моделей:")
    print(evaluation_summary.head(20))
    
    # Визуализация метрик
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # MAE по моделям
    mae_by_model = evaluation_summary.groupby('model_name')['mae'].mean().sort_values()
    axes[0, 0].barh(range(len(mae_by_model)), mae_by_model.values)
    axes[0, 0].set_yticks(range(len(mae_by_model)))
    axes[0, 0].set_yticklabels(mae_by_model.index)
    axes[0, 0].set_title('Средний MAE по моделям')
    axes[0, 0].set_xlabel('MAE')
    
    # MAPE по моделям
    mape_by_model = evaluation_summary.groupby('model_name')['mape'].mean().sort_values()
    axes[0, 1].barh(range(len(mape_by_model)), mape_by_model.values)
    axes[0, 1].set_yticks(range(len(mape_by_model)))
    axes[0, 1].set_yticklabels(mape_by_model.index)
    axes[0, 1].set_title('Средний MAPE по моделям')
    axes[0, 1].set_xlabel('MAPE (%)')
    
    # R² по моделям
    r2_by_model = evaluation_summary.groupby('model_name')['r2'].mean().sort_values(ascending=False)
    axes[1, 0].barh(range(len(r2_by_model)), r2_by_model.values)
    axes[1, 0].set_yticks(range(len(r2_by_model)))
    axes[1, 0].set_yticklabels(r2_by_model.index)
    axes[1, 0].set_title('Средний R² по моделям')
    axes[1, 0].set_xlabel('R²')
    
    # Распределение лучших моделей
    if not best_models_summary.empty:
        best_models_dist = best_models_summary['best_model'].value_counts()
        axes[1, 1].pie(best_models_dist.values, labels=best_models_dist.index, autopct='%1.1f%%')
        axes[1, 1].set_title('Распределение лучших моделей')
    
    plt.tight_layout()
    plt.show()

if not best_models_summary.empty:
    print("\nЛучшие модели по продуктам:")
    print(best_models_summary.head(20))


## 10. Сохранение результатов


In [ ]:
# Сохранение результатов
output_dir = Path('../output')
output_dir.mkdir(exist_ok=True)

# Прогнозы продаж
wb_best_forecast.to_csv(output_dir / 'wb_forecast.csv', index=False, encoding='utf-8-sig')
ozon_best_forecast.to_csv(output_dir / 'ozon_forecast.csv', index=False, encoding='utf-8-sig')
print("Прогнозы продаж сохранены")

# Отгрузки
if not wb_shipments.empty:
    wb_shipments.to_csv(output_dir / 'wb_shipments.csv', index=False, encoding='utf-8-sig')
    print("Отгрузки WB сохранены")
if not ozon_shipments.empty:
    ozon_shipments.to_csv(output_dir / 'ozon_shipments.csv', index=False, encoding='utf-8-sig')
    print("Отгрузки Ozon сохранены")

# Оценка моделей
if not evaluation_summary.empty:
    evaluation_summary.to_csv(output_dir / 'model_evaluation.csv', index=False, encoding='utf-8-sig')
    print("Оценка моделей сохранена")
if not best_models_summary.empty:
    best_models_summary.to_csv(output_dir / 'best_models.csv', index=False, encoding='utf-8-sig')
    print("Лучшие модели сохранены")

print(f"\nВсе результаты сохранены в папку {output_dir}")


## 11. Пример анализа конкретного продукта


In [ ]:
# Выберите продукт для анализа
if not wb_best_forecast.empty:
    product_code = wb_best_forecast['unified_code'].iloc[0]
    print(f"Анализ продукта: {product_code}")
    
    # Исторические продажи
    if 'wb_sales' in data and not data['wb_sales'].empty:
        product_sales = data['wb_sales'][data['wb_sales']['unified_code'] == product_code].copy()
        product_sales = product_sales.sort_values('date')
        
        # Прогноз
        product_forecast = wb_best_forecast[wb_best_forecast['unified_code'] == product_code].copy()
        product_forecast = product_forecast.sort_values('date')
        
        # Визуализация
        fig, ax = plt.subplots(figsize=(15, 6))
        
        # Исторические продажи
        ax.plot(product_sales['date'], product_sales['quantity'], 
                label='Исторические продажи', marker='o', alpha=0.7)
        
        # Прогноз
        ax.plot(product_forecast['date'], product_forecast['quantity'], 
                label='Прогноз', marker='s', linestyle='--', alpha=0.7)
        
        ax.set_title(f'Продажи и прогноз для продукта {product_code}')
        ax.set_xlabel('Дата')
        ax.set_ylabel('Количество упаковок')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Выбранная модель
        selected_model = product_forecast['selected_model'].iloc[0] if 'selected_model' in product_forecast.columns else 'N/A'
        print(f"Выбранная модель: {selected_model}")
